# SE370 Lesson 10 — Pandas DataFrames (Lecture + Exercise)

## Learning objectives
By the end of this lesson, you should be able to:
1. Import and use **pandas**.
2. Create DataFrames from dictionaries, lists, and a list of dictionaries.
3. Read CSV/Excel/JSON into DataFrames.
4. Index/subset rows and columns using `[]`, `.loc`, and `.iloc`.
5. Rename columns and add/drop columns.
6. Save DataFrames to files.
7. Loop over DataFrames with `.iterrows()` (and understand when not to).

---


## 0) Setup

If you see an error like `ModuleNotFoundError: No module named 'pandas'`, install pandas:
- `pip install pandas`

For Excel support, you may also need:

- `pip install openpyxl`



In [1]:
import pandas as pd

# Display more columns/rows in notebook outputs (optional)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)


## 1) Creating DataFrames

### 1.1 Create a DataFrame directly (dictionary of lists)


In [2]:
df = pd.DataFrame({'name': ['ian', 'john'], 'department': ['DSE', 'Math']})
df


,name,department
0,ian,DSE
1,john,Math


### 1.2 Create a DataFrame from existing lists

This makes the connection between **Python lists** and **DataFrames** concrete.


In [3]:
name = ['ian', 'john']
dept = ['DSE', 'Math']

df = pd.DataFrame({'name': name, 'dept': dept})
df


,name,dept
0,ian,DSE
1,john,Math


### 1.3 Create a DataFrame from a list of dictionaries

This pattern shows up often when data comes from APIs or JSON.


In [4]:
data = [
    {'name': 'ian', 'dept': 'DSE'},
    {'name': 'john', 'dept': 'Math'}
]

df = pd.DataFrame(data)
df


,name,dept
0,ian,DSE
1,john,Math


## 2) Reading DataFrames from files

In class, you may have files like `sample.csv`, `sample.xlsx`, and `sample.json`.

✅ **Best practice for class notebooks:** Use **relative paths** so the notebook works on any computer *when files are in the same folder as the notebook*.

> This cell sets the path for three of the exercise data files. This is currently written as if you have all the exercise files in a folder called **data** that sits at the same level as this notebook. Adjust as needed.


In [5]:
csv_path = 'data/sample.csv'
xlsx_path = 'data/sample.xlsx'
json_path = 'data/sample.json'

This cell reads in the csv.

In [6]:
pd.read_csv(csv_path)

,Name,Age,City
0,Alice,30,New York
1,Bob,25,San Francisco
2,Charlie,35,Chicago


This cell reads in the excel (xlsx) file. If you don't have it you might need to install openpyxl.

In [7]:
pd.read_excel(xlsx_path)

,Name,Age,City
0,Alice,30,New York
1,Bob,25,San Francisco
2,Charlie,35,Chicago


This cell reads in the json.

In [8]:
pd.read_json(json_path)

,Name,Age,City
0,Alice,30,New York
1,Bob,25,San Francisco
2,Charlie,35,Chicago


## 3) Basic DataFrame manipulation (birds dataset)

- Load `exercise.csv`

In [9]:
# Where is your exercise.csv file?
df = pd.read_csv('')
   
df.head()

FileNotFoundError: [Errno 2] No such file or directory: ''

### 3.1 How big is the DataFrame?

- `len(df)` → number of rows
- `len(df.columns)` → number of columns
- `df.shape` → (rows, columns)


In [ ]:
len(df), len(df.columns), df.shape


### 3.2 Rename columns

You can rename by assigning to `df.columns` (must match number of columns).


In [ ]:
df.columns = ['id','species','color','habitat','wingspan']
df.head()


### 3.3 Column selection

- Single column: `df['species']`
- Multiple columns: `df[['species', 'wingspan']]`
- Convert a column to a list: `list(df['species'])`


In [ ]:
df['species'].head()


In [ ]:
df[['species','wingspan']].head()

In [ ]:
list(df['species'])

### 3.4 Drop columns

Use `axis=1` to drop a column.


In [ ]:
df = df.drop('id', axis=1)
df.head()


### 3.5 Add new columns

Two common patterns:

1. Bulk-assign a constant using `.assign()`
2. Create a computed column with vectorized math (fast!)


In [ ]:
df = df.assign(animal_type='bird')

# Vectorized math: divide wingspan by 10
df['wingspan_divided'] = df['wingspan'] / 10

df

In [ ]:
df.dtypes

In [ ]:
# Convert the computed column to integers

df['wingspan_divided'] = df['wingspan_divided'].astype('int')
df.head()


## 4) Index operations (`set_index`, `reset_index`)

You *can* set the index to a meaningful label (like `species`) to make row lookup easy.
We will demonstrate it, then reset back.


In [ ]:
df = df.set_index('species')
df.head()


In [ ]:
# Example row lookup by index label

if 'Hummingbird' in df.index:
    display(df.loc['Hummingbird'])
else:
    print("Hummingbird not in this dataset.")


In [ ]:
# Move index back into a regular column

df = df.reset_index()
df.head()


## 5) Row subsetting (`head`, slicing, `.loc`, `.iloc`)

- `df[:10]` → first 10 rows
- `.loc` is label-based and **inclusive** on slices
- `.iloc` is position-based and **exclusive** on the end of slices

<img src="image.png" alt="loc vs iloc" width="500">

In [ ]:
df_10 = df[:10]
df_10

In [ ]:
df.loc[1:3]

In [ ]:
df.loc[1:3, ['color','habitat']]

To get the same thing with iloc you use numbers to call the rows and columns. **Notice the exclusion in iloc vs loc**

In [ ]:
df.iloc[1:4, 1:3]

## 6) Save a DataFrame to CSV

Most of the time you'll save as **CSV**.

> Tip: Usually you do **not** want to save the index (`index=False`).


In [ ]:
df.to_csv('data/test_save.csv', index=False) #Saves to the data folder in your current working directory

## 7) Looping over a DataFrame (`iterrows`)

Looping works, but **vectorized operations are usually faster**.

- This format allows you to use both the row and the index


In [ ]:
for index, row in df.head(3).iterrows():
    print(index)

In [ ]:
for index, row in df.head(1).iterrows():
    print(row)

We'll loop to create a boolean column `yellow` that is `True` when the color is Yellow.

In [ ]:
df = df.assign(yellow=False)

for index, row in df.iterrows():
    if row['color'] == 'Yellow':
        df.loc[index, 'yellow'] = True

df[['color', 'yellow']].head(10)

### (Optional) Vectorized alternative

This is the preferred pandas style (no Python loop):

```python
df['yellow'] = df['color'].eq('Yellow')
```

- df['color'] returns the Series (one column) containing color values like "Yellow", "Blue", etc.
- .eq('Yellow') means “equals Yellow” applied to every element in that Series.
- The result is a boolean Series (True/False) of the same length as df.


In [ ]:
df['yellow_vectorized'] = df['color'].eq('Yellow')
df[['species','color','yellow','yellow_vectorized']].head(10)

## 8) Exercise — Electric cars

**Task:**
1. Read `electric_cars.csv`
2. Remove the column containing the manufacturer (`make`)
3. Convert top speed to mph (it is in kph currently)
4. Create a column called `price_format` that is a string with a `$` in front of the price (use a loop)
5. Save as `electric_cars_new.csv`

This notebook includes a fallback dataset so the exercise runs even if the file is not present.


In [ ]:
# Read in electric_cars.csv
cars = pd.read_csv('')
   
cars.head(5)


In [ ]:
# 1) Remove manufacturer column
cars = cars.drop('', axis=)

cars.head()

In [ ]:
# 2) Convert speed from kph to mph
# 1 mile = 1.609 km  => mph = kph / 1.609

cars['speed_mph'] = 

cars.head()

In [ ]:
# 3) Create price_format using a loop
price_format = []
for index, row in cars.iterrows():
    price_format.append('$' + str(### Adjust this portion ###))

cars = cars.assign(price_format=price_format)
cars


In [ ]:
# 4) Save as electric_cars_new.csv
cars.to_csv('', index=)


## 9) Stretch Exercise — Add a comma to the price string

**Without external packages**, insert a comma before the last 3 digits.

Example:
- `$38990` → `$38,990`

We'll do this with string slicing.


In [ ]:
new_col = []
for index, row in cars.iterrows():
    s = row['price_format']
    # Handle short prices safely
    if len(s) > ######:
        tmp = s[######] + ',' + s[#######]
    else:
        tmp = ########
    new_col.append(tmp)

cars = cars.assign(price_format=new_col)
cars


---

## Notes for students
- Prefer **vectorized pandas operations** when possible.
- Use **relative file paths** to keep notebooks portable.
- Save with `index=False` unless you explicitly want the index in the file.

